In [1]:
import numpy as np
import pylab as plt

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
seq = keras.Sequential(
    [
        keras.Input(
            shape = (None, 40, 40, 1)
        ),  
        layers.ConvLSTM2D(
            filters = 40, kernel_size = (3, 3), padding = "same", return_sequences = True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters = 40, kernel_size = (3, 3), padding = "same", return_sequences = True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters = 40, kernel_size = (3, 3), padding = "same", return_sequences = True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters = 40, kernel_size = (3, 3), padding = "same", return_sequences = True
        ),
        layers.BatchNormalization(),
        layers.Conv3D(
            filters = 1, kernel_size = (3, 3, 3), activation = "sigmoid", padding = "same"
        ),
    ]
)

seq.compile(loss = "binary_crossentropy", optimizer = "adadelta")

In [3]:
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, None, 40, 40, 40)  59200     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 40, 40, 40)  115360    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 40, 40, 40)  115360    
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, None, 40, 40, 40)  1